<a href="https://colab.research.google.com/github/Tom-Jung/Tom-Jung/blob/main/API_DART_F_%EC%83%81%EC%9E%A5%ED%9A%8C%EC%82%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install dart-fss

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.5/147.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.1/201.1 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 MB 8.6 MB/s eta 0:00:00
  Attempting uninstall: termcolor
    Found existing installation: termcolor 2.5.0
    Uninstalling termcolor-2.5.0:
      Successfully uninstalled termcolor-2.5.0


In [ ]:
import requests
import pandas as pd

# DART API 인증키
API_KEY = '85006d20472a255d5eae25ee265b5ed1b78d1386'  # 자신의 인증키로 변경하세요

# 재무제표 API URL
FS_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

# CSV 파일에서 기업리스트 읽기
# 기업 리스트 파일은 '기업리스트.csv'라는 이름으로 가정하며 'code'라는 열을 포함합니다
corporations_df = pd.read_excel("/content/corp_list_split.xlsx", sheet_name="L1",  dtype={'code': str})

# 데이터 저장을 위한 리스트
financial_data = []

# 각 기업에 대해 재무제표 데이터 요청
for _, row in corporations_df.iterrows():
    corp_code = row['code']  # 'code' 열 사용
    corp_name = row['name'] if 'name' in row else '알 수 없음'  # 'name' 열이 있는 경우 사용

    params = {
        'crtfc_key': API_KEY,
        'corp_code': corp_code,
        'bsns_year': 2023,  # 연도 예시
        'reprt_code': '11011',  # 연결 재무제표
        'fs_div': 'OFS'  # 개별 재무제표 (OFS)
    }

    # DART API 요청
    response = requests.get(FS_URL, params=params)

    if response.status_code == 200:
        data = response.json()

        # 데이터 출력
        print(f"기업 {corp_name}({corp_code})의 응답 데이터: {data}")

        # 재무제표 데이터 처리
        if 'list' in data and data['list']:
            for item in data['list']:
                financial_data.append({
                    '기업 이름': corp_name,
                    '기업 코드': corp_code,
                    '재무제표 항목': item['account_nm'],
                    '2023년': item.get('thstrm_amount', '데이터 없음'),
                    '2022년': item.get('frmtrm_amount', '데이터 없음'),
                    '2021년': item.get('lfy_amount', '데이터 없음')
                })
        else:
            print(f"기업 {corp_name}({corp_code})의 재무제표 데이터가 없습니다.")
    else:
        print(f"기업 {corp_name}({corp_code})에 대한 API 요청 실패: {response.status_code}")

# 결과를 DataFrame으로 변환
df_1 = pd.DataFrame(financial_data)

# 결과를 엑셀 파일로 저장
df_1.to_excel("기업_재무제표_2023년_L1.xlsx", index=False)
print("기업 재무제표 데이터_L1_를 엑셀 파일로 저장했습니다.")

#
:------------------------------------------------------------------------------L2------------------------------------

In [ ]:
import requests
import pandas as pd

# DART API 인증키
API_KEY = '85006d20472a255d5eae25ee265b5ed1b78d1386'  # 자신의 인증키로 변경하세요

# 재무제표 API URL
FS_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

# CSV 파일에서 기업리스트 읽기
# 기업 리스트 파일은 '기업리스트.csv'라는 이름으로 가정하며 'code'라는 열을 포함합니다
corporations_df = pd.read_excel("/content/corp_list_split.xlsx", sheet_name="L2",  dtype={'code': str})

# 데이터 저장을 위한 리스트
financial_data = []

# 각 기업에 대해 재무제표 데이터 요청
for _, row in corporations_df.iterrows():
    corp_code = row['code']  # 'code' 열 사용
    corp_name = row['name'] if 'name' in row else '알 수 없음'  # 'name' 열이 있는 경우 사용

    params = {
        'crtfc_key': API_KEY,
        'corp_code': corp_code,
        'bsns_year': 2023,  # 연도 예시
        'reprt_code': '11011',  # 연결 재무제표
        'fs_div': 'OFS'  # 개별 재무제표 (OFS)
    }

    # DART API 요청
    response = requests.get(FS_URL, params=params)

    if response.status_code == 200:
        data = response.json()

        # 데이터 출력
        print(f"기업 {corp_name}({corp_code})의 응답 데이터: {data}")

        # 재무제표 데이터 처리
        if 'list' in data and data['list']:
            for item in data['list']:
                financial_data.append({
                    '기업 이름': corp_name,
                    '기업 코드': corp_code,
                    '재무제표 항목': item['account_nm'],
                    '2023년': item.get('thstrm_amount', '데이터 없음'),
                    '2022년': item.get('frmtrm_amount', '데이터 없음'),
                    '2021년': item.get('lfy_amount', '데이터 없음')
                })
        else:
            print(f"기업 {corp_name}({corp_code})의 재무제표 데이터가 없습니다.")
    else:
        print(f"기업 {corp_name}({corp_code})에 대한 API 요청 실패: {response.status_code}")

# 결과를 DataFrame으로 변환
df_2 = pd.DataFrame(financial_data)

# 결과를 엑셀 파일로 저장
df_2.to_excel("기업_재무제표_2023년_L2.xlsx", index=False)
print("기업 재무제표 데이터_L2_를 엑셀 파일로 저장했습니다.")

#
:------------------------------------------------------------------------------L3------------------------------------




In [ ]:
import requests
import pandas as pd

# DART API 인증키
API_KEY = '85006d20472a255d5eae25ee265b5ed1b78d1386'  # 자신의 인증키로 변경하세요

# 재무제표 API URL
FS_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

# CSV 파일에서 기업리스트 읽기
# 기업 리스트 파일은 '기업리스트.csv'라는 이름으로 가정하며 'code'라는 열을 포함합니다
corporations_df = pd.read_excel("/content/corp_list_split.xlsx", sheet_name="L3",  dtype={'code': str})

# 데이터 저장을 위한 리스트
financial_data = []

# 각 기업에 대해 재무제표 데이터 요청
for _, row in corporations_df.iterrows():
    corp_code = row['code']  # 'code' 열 사용
    corp_name = row['name'] if 'name' in row else '알 수 없음'  # 'name' 열이 있는 경우 사용

    params = {
        'crtfc_key': API_KEY,
        'corp_code': corp_code,
        'bsns_year': 2023,  # 연도 예시
        'reprt_code': '11011',  # 연결 재무제표
        'fs_div': 'OFS'  # 개별 재무제표 (OFS)
    }

    # DART API 요청
    response = requests.get(FS_URL, params=params)

    if response.status_code == 200:
        data = response.json()

        # 데이터 출력
        print(f"기업 {corp_name}({corp_code})의 응답 데이터: {data}")

        # 재무제표 데이터 처리
        if 'list' in data and data['list']:
            for item in data['list']:
                financial_data.append({
                    '기업 이름': corp_name,
                    '기업 코드': corp_code,
                    '재무제표 항목': item['account_nm'],
                    '2023년': item.get('thstrm_amount', '데이터 없음'),
                    '2022년': item.get('frmtrm_amount', '데이터 없음'),
                    '2021년': item.get('lfy_amount', '데이터 없음')
                })
        else:
            print(f"기업 {corp_name}({corp_code})의 재무제표 데이터가 없습니다.")
    else:
        print(f"기업 {corp_name}({corp_code})에 대한 API 요청 실패: {response.status_code}")

# 결과를 DataFrame으로 변환
df_3 = pd.DataFrame(financial_data)

# 결과를 엑셀 파일로 저장
df_3.to_excel("기업_재무제표_2023년_L3.xlsx", index=False)
print("기업 재무제표 데이터_L3_를 엑셀 파일로 저장했습니다.")

#
:------------------------------------------------------------------------------L4------------------------------------

In [ ]:
import requests
import pandas as pd

# DART API 인증키
API_KEY = '85006d20472a255d5eae25ee265b5ed1b78d1386'  # 자신의 인증키로 변경하세요

# 재무제표 API URL
FS_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

# CSV 파일에서 기업리스트 읽기
# 기업 리스트 파일은 '기업리스트.csv'라는 이름으로 가정하며 'code'라는 열을 포함합니다
corporations_df = pd.read_excel("/content/corp_list_split.xlsx", sheet_name="L4",  dtype={'code': str})

# 데이터 저장을 위한 리스트
financial_data = []

# 각 기업에 대해 재무제표 데이터 요청
for _, row in corporations_df.iterrows():
    corp_code = row['code']  # 'code' 열 사용
    corp_name = row['name'] if 'name' in row else '알 수 없음'  # 'name' 열이 있는 경우 사용

    params = {
        'crtfc_key': API_KEY,
        'corp_code': corp_code,
        'bsns_year': 2023,  # 연도 예시
        'reprt_code': '11011',  # 연결 재무제표
        'fs_div': 'OFS'  # 개별 재무제표 (OFS)
    }

    # DART API 요청
    response = requests.get(FS_URL, params=params)

    if response.status_code == 200:
        data = response.json()

        # 데이터 출력
        print(f"기업 {corp_name}({corp_code})의 응답 데이터: {data}")

        # 재무제표 데이터 처리
        if 'list' in data and data['list']:
            for item in data['list']:
                financial_data.append({
                    '기업 이름': corp_name,
                    '기업 코드': corp_code,
                    '재무제표 항목': item['account_nm'],
                    '2023년': item.get('thstrm_amount', '데이터 없음'),
                    '2022년': item.get('frmtrm_amount', '데이터 없음'),
                    '2021년': item.get('lfy_amount', '데이터 없음')
                })
        else:
            print(f"기업 {corp_name}({corp_code})의 재무제표 데이터가 없습니다.")
    else:
        print(f"기업 {corp_name}({corp_code})에 대한 API 요청 실패: {response.status_code}")

# 결과를 DataFrame으로 변환
df_4 = pd.DataFrame(financial_data)

# 결과를 엑셀 파일로 저장
df_4.to_excel("기업_재무제표_2023년_L4.xlsx", index=False)
print("기업 재무제표 데이터_L4_를 엑셀 파일로 저장했습니다.")

#
:------------------------------------------------------------------------------L5------------------------------------

In [ ]:
import requests
import pandas as pd

# DART API 인증키
API_KEY = '85006d20472a255d5eae25ee265b5ed1b78d1386'  # 자신의 인증키로 변경하세요

# 재무제표 API URL
FS_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

# CSV 파일에서 기업리스트 읽기
# 기업 리스트 파일은 '기업리스트.csv'라는 이름으로 가정하며 'code'라는 열을 포함합니다
corporations_df = pd.read_excel("/content/corp_list_split.xlsx", sheet_name="L5",  dtype={'code': str})

# 데이터 저장을 위한 리스트
financial_data = []

# 각 기업에 대해 재무제표 데이터 요청
for _, row in corporations_df.iterrows():
    corp_code = row['code']  # 'code' 열 사용
    corp_name = row['name'] if 'name' in row else '알 수 없음'  # 'name' 열이 있는 경우 사용

    params = {
        'crtfc_key': API_KEY,
        'corp_code': corp_code,
        'bsns_year': 2023,  # 연도 예시
        'reprt_code': '11011',  # 연결 재무제표
        'fs_div': 'OFS'  # 개별 재무제표 (OFS)
    }

    # DART API 요청
    response = requests.get(FS_URL, params=params)

    if response.status_code == 200:
        data = response.json()

        # 데이터 출력
        print(f"기업 {corp_name}({corp_code})의 응답 데이터: {data}")

        # 재무제표 데이터 처리
        if 'list' in data and data['list']:
            for item in data['list']:
                financial_data.append({
                    '기업 이름': corp_name,
                    '기업 코드': corp_code,
                    '재무제표 항목': item['account_nm'],
                    '2023년': item.get('thstrm_amount', '데이터 없음'),
                    '2022년': item.get('frmtrm_amount', '데이터 없음'),
                    '2021년': item.get('lfy_amount', '데이터 없음')
                })
        else:
            print(f"기업 {corp_name}({corp_code})의 재무제표 데이터가 없습니다.")
    else:
        print(f"기업 {corp_name}({corp_code})에 대한 API 요청 실패: {response.status_code}")

# 결과를 DataFrame으로 변환
df_5 = pd.DataFrame(financial_data)

# 결과를 엑셀 파일로 저장
df_5.to_excel("기업_재무제표_2023년_L5.xlsx", index=False)
print("기업 재무제표 데이터_L5_를 엑셀 파일로 저장했습니다.")

In [12]:
df_fn = pd.concat([df_1,df_2,df_3,df_4,df_5])
df_fn.to_excel('상장법인재무제표.xlsx')

In [ ]:
import requests
import pandas as pd

# DART API 인증키
API_KEY = '85006d20472a255d5eae25ee265b5ed1b78d1386'  # 본인의 인증키로 변경

# 삼성전자 기업 코드 (DART 고유 코드)
samsung_code = "00126380"

# 재무제표 API URL
FS_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

# 데이터 저장을 위한 리스트
financial_data = []

# API 요청 파라미터
params = {
    'crtfc_key': API_KEY,
    'corp_code': samsung_code,
    'bsns_year': 2023,  # 연도
    'reprt_code': '11011',  # 연결 재무제표
    'fs_div': 'OFS'  # 개별 재무제표
}

# DART API 요청
response = requests.get(FS_URL, params=params)

if response.status_code == 200:
    data = response.json()

    # 데이터 출력
    print(f"삼성전자({samsung_code})의 응답 데이터: {data}")

    # 재무제표 데이터 처리
    if 'list' in data and data['list']:
        for item in data['list']:
            financial_data.append({
                '기업 이름': '삼성전자',
                '기업 코드': samsung_code,
                '재무제표 항목': item['account_nm'],
                '2023년': item.get('thstrm_amount', '데이터 없음'),
                '2022년': item.get('frmtrm_amount', '데이터 없음'),
                '2021년': item.get('lfy_amount', '데이터 없음')
            })
    else:
        print(f"삼성전자({samsung_code})의 재무제표 데이터가 없습니다.")
else:
    print(f"삼성전자({samsung_code})에 대한 API 요청 실패: {response.status_code}")

# 결과를 DataFrame으로 변환
df_samsung = pd.DataFrame(financial_data)

# 결과를 엑셀 파일로 저장
df_samsung.to_excel("삼성전자_재무제표_2023년.xlsx", index=False)
print("삼성전자 재무제표 데이터를 엑셀 파일로 저장했습니다.")


In [20]:
import requests
import pandas as pd

# DART API 인증키
API_KEY = '85006d20472a255d5eae25ee265b5ed1b78d1386'  # 본인의 인증키로 변경

# 삼성전자 기업 코드 (DART 고유 코드)
samsung_code = "00126380"

# 재무제표 API URL
FS_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

# 개별 재무제표 요청 (OFS)
params = {
    'crtfc_key': API_KEY,
    'corp_code': samsung_code,
    'bsns_year': 2024,  # 연도
    'reprt_code': '11012',  # 개별 재무제표
    'fs_div': 'OFS'  # 개별 재무제표
}

# DART API 요청
response = requests.get(FS_URL, params=params)

# 응답이 정상적인 경우 데이터 처리
if response.status_code == 200:
    data = response.json()

    # 응답 데이터 확인
    if 'list' in data and data['list']:
        df = pd.DataFrame(data['list'])

        # 필요한 컬럼이 모두 있는지 확인 후, 없으면 기본값 추가
        for col in ['account_nm', 'thstrm_amount', 'frmtrm_amount', 'lfy_amount']:
            if col not in df.columns:
                df[col] = '데이터 없음'  # 값이 없을 경우 기본값 설정

        # 재무제표 종류별 구분 코드 (sj_div)
        financial_statements = {
            "BS": "재무상태표",
            "IS": "손익계산서",
            "CIS": "포괄손익계산서",
            "CF": "현금흐름표",
            "SCE": "자본변동표"
        }

        # 각 재무제표별로 분리 후 저장
        for sj_div, sheet_name in financial_statements.items():
            df_filtered = df[df['sj_div'] == sj_div]

            if not df_filtered.empty:
                # 필요한 컬럼만 선택
                df_filtered = df_filtered[['account_nm', 'thstrm_amount', 'frmtrm_amount', 'lfy_amount']]
                df_filtered.columns = ['재무제표 항목', '2023년', '2022년', '2021년']

                # 엑셀 저장
                file_name = f"삼성전자_{sheet_name}_2023년.xlsx"
                df_filtered.to_excel(file_name, index=False)
                print(f"{sheet_name} 데이터를 엑셀 파일로 저장했습니다: {file_name}")
            else:
                print(f"{sheet_name} 데이터가 없습니다.")
    else:
        print(f"삼성전자({samsung_code})의 개별 재무제표 데이터가 없습니다.")
else:
    print(f"삼성전자({samsung_code}) 개별 재무제표 API 요청 실패: {response.status_code}")


재무상태표 데이터를 엑셀 파일로 저장했습니다: 삼성전자_재무상태표_2023년.xlsx
손익계산서 데이터를 엑셀 파일로 저장했습니다: 삼성전자_손익계산서_2023년.xlsx
포괄손익계산서 데이터를 엑셀 파일로 저장했습니다: 삼성전자_포괄손익계산서_2023년.xlsx
현금흐름표 데이터를 엑셀 파일로 저장했습니다: 삼성전자_현금흐름표_2023년.xlsx
자본변동표 데이터를 엑셀 파일로 저장했습니다: 삼성전자_자본변동표_2023년.xlsx


In [24]:
import requests
import pandas as pd

# DART API 인증키 (본인의 인증키로 변경)
API_KEY = '85006d20472a255d5eae25ee265b5ed1b78d1386'  # 자신의 인증키로 변경

# 삼성전자 고유 코드
samsung_code = "00126380"

# 재무제표 API URL
FS_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

# 개별 재무제표 요청 (OFS)
params = {
    'crtfc_key': API_KEY,
    'corp_code': samsung_code,
    'bsns_year': 2023,  # 2022년
    'reprt_code': '11012',  # 개별 재무제표
    'fs_div': 'OFS'  # 개별 재무제표
}

# DART API 요청
response = requests.get(FS_URL, params=params)

# 응답이 정상적인 경우 데이터 처리
if response.status_code == 200:
    data = response.json()

    # 응답 데이터 확인
    if 'list' in data and data['list']:
        df = pd.DataFrame(data['list'])

        # 필요한 컬럼이 모두 있는지 확인 후, 없으면 기본값 추가
        for col in ['account_nm', 'thstrm_amount', 'frmtrm_amount', 'lfy_amount']:
            if col not in df.columns:
                df[col] = '데이터 없음'  # 값이 없을 경우 기본값 설정

        # 숫자 변환
        df['thstrm_amount'] = pd.to_numeric(df['thstrm_amount'], errors='coerce')
        df['frmtrm_amount'] = pd.to_numeric(df['frmtrm_amount'], errors='coerce')
        df['lfy_amount'] = pd.to_numeric(df['lfy_amount'], errors='coerce')

        # NaN 값 확인
        print(df.isnull().sum())

        # 재무제표 종류별 구분 코드 (sj_div)
        financial_statements = {
            "BS": "재무상태표",
            "IS": "손익계산서",
            "CIS": "포괄손익계산서",
            "CF": "현금흐름표",
            "SCE": "자본변동표"
        }

        # 각 재무제표별로 분리 후 저장
        for sj_div, sheet_name in financial_statements.items():
            df_filtered = df[df['sj_div'] == sj_div]

            if not df_filtered.empty:
                # 필요한 컬럼만 선택
                df_filtered = df_filtered[['account_nm', 'thstrm_amount', 'frmtrm_amount', 'lfy_amount']]
                df_filtered.columns = ['재무제표 항목', '2023년', '2022년', '2021년']

                # 엑셀 저장
                file_name = f"삼성전자_{sheet_name}_2023년.xlsx"
                df_filtered.to_excel(file_name, index=False)
                print(f"{sheet_name} 데이터를 엑셀 파일로 저장했습니다: {file_name}")



rcept_no               0
reprt_code             0
bsns_year              0
corp_code              0
sj_div                 0
sj_nm                  0
account_id             0
account_nm             0
account_detail         0
thstrm_nm              0
thstrm_amount          1
frmtrm_nm             68
frmtrm_amount         68
ord                    0
currency               0
thstrm_add_amount     91
frmtrm_q_nm           44
frmtrm_q_amount       46
frmtrm_add_amount     91
lfy_amount           112
dtype: int64
재무상태표 데이터를 엑셀 파일로 저장했습니다: 삼성전자_재무상태표_2023년.xlsx
손익계산서 데이터를 엑셀 파일로 저장했습니다: 삼성전자_손익계산서_2023년.xlsx
포괄손익계산서 데이터를 엑셀 파일로 저장했습니다: 삼성전자_포괄손익계산서_2023년.xlsx
현금흐름표 데이터를 엑셀 파일로 저장했습니다: 삼성전자_현금흐름표_2023년.xlsx
자본변동표 데이터를 엑셀 파일로 저장했습니다: 삼성전자_자본변동표_2023년.xlsx
